# aptos submission

In [1]:
from pathlib import Path
from tensorflow.keras import models
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image

import os

import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
PROJECT_DIR = Path('/notebooks/storage/aptos')
TEST_DIR = PROJECT_DIR/'images'/'test'
TEST_CSV = PROJECT_DIR/'test.csv'

FILENAME = 'model_a.h5'

os.chdir(str(PROJECT_DIR))

### load model

In [3]:
%%time

model = models.load_model(str(PROJECT_DIR/FILENAME))

CPU times: user 9.26 s, sys: 1.53 s, total: 10.8 s
Wall time: 10.7 s


### load test csv

In [4]:
%%time

test_df = pd.read_csv(TEST_CSV)

CPU times: user 10.9 ms, sys: 0 ns, total: 10.9 ms
Wall time: 12.4 ms


In [5]:
test_df.tail()

,id_code
1923,ff2fd94448de
1924,ff4c945d9b17
1925,ff64897ac0d8
1926,ffa73465b705
1927,ffdc2152d455


### generate predictions

In [11]:
%%time

guesses = []

for index,row in test_df.iterrows():
    
    fname = row['id_code']+'.png'
    
    img_path = TEST_DIR/fname
    
    img = image.load_img(str(img_path),target_size=(150,150))
    
    x = image.img_to_array(img)/255.
    
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    
    classes = model.predict(images, batch_size=10)
    
    guesses += [classes[0][0]]
    
    if index%10 == 0:
        print(index)

guesses




0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
CPU times: user 2min 20s, sys: 12.7 s, total: 2min 33s
Wall time: 2min 37s


In [12]:
%%time

rounded_guesses = []

for index, pred in enumerate(guesses):
    
    rounded_guesses += [int(round(pred))]

test_df['diagnosis'] = pd.Series(rounded_guesses)

CPU times: user 6.64 ms, sys: 4.27 ms, total: 10.9 ms
Wall time: 10.1 ms


In [13]:
test_df.tail()

,id_code,diagnosis
1923,ff2fd94448de,0
1924,ff4c945d9b17,0
1925,ff64897ac0d8,1
1926,ffa73465b705,2
1927,ffdc2152d455,0


In [14]:
test_df.describe()

,diagnosis
count,1928.000000
mean,0.691390
std,0.764881
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,3.000000


### write to csv

In [15]:
os.chdir(str(PROJECT_DIR))

test_df.to_csv('submission.csv',index=False)